In [13]:
import requests
import pandas as pd
from db import create_table, insert_data
from etl import get_raw_data_colums
colums = set()

In [15]:
URL = 'http://10.43.100.103:8000/data?group_number=4&day=Tuesday'
RESTART_URL = 'http://10.43.100.103:8000/restart_data_generation?group_number=4&day=Tuesday'

In [14]:
def fetch_data():
    
    response = requests.get(URL)
    print("fetch data is starting...")
    columns = set()
    while (response.status_code == 200):
        first_time = False
        print(response)
        data = response.json()['data']
        batch_number = response.json()['batch_number']
        print(f"✅ Batch number {batch_number}")
        print(f"✅ Data {len(data)}")
        df = pd.DataFrame(data)
        print(f"✅ DF {df.head()}")
        print(f" Columnas: {df.columns}")
        columns.update(df.columns)
        response = requests.get(URL)
    print("after while")
    df_columns = pd.DataFrame(columns=["columns"])
    create_table("raw_data_columns", df_columns)
    formatted_columns = pd.DataFrame(list(columns), columns=["columns"])
    insert_data("raw_data_columns", formatted_columns)
    restart_response = requests.get(RESTART_URL)
    print(f"All data fetched. Columns saved, restart response: {restart_response}")

In [15]:
fetch_data()

fetch data is starting...
<Response [200]>
✅ Batch number 0
✅ Data 73784
✅ DF    brokered_by    status     price  bed  bath  acre_lot     street  \
0     101640.0  for_sale  289900.0  4.0   2.0      0.38  1758218.0   
1     107951.0  for_sale  299900.0  3.0   2.0      0.87  1336295.0   
2      80935.0  for_sale  299000.0  3.0   2.0      0.35   920059.0   
3      33714.0  for_sale  221000.0  4.0   2.0      0.32   731702.0   
4      29997.0  for_sale  175000.0  3.0   2.0      0.19  1382878.0   

            city        state  zip_code  house_size prev_sold_date  
0   East Windsor  Connecticut    6016.0      1617.0     1999-09-30  
1         Vernon  Connecticut    6066.0      1850.0     2015-11-09  
2   North Canaan  Connecticut    6018.0      1620.0     2011-08-23  
3  Windsor Locks  Connecticut    6096.0      1735.0     2014-03-03  
4     Winchester  Connecticut    6098.0      2005.0     2007-07-19  
 Columnas: Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street'

In [9]:
column_names = get_raw_data_colums()
df_new = pd.DataFrame(columns=column_names["columns"].tolist())

print(df_new)


port 8005
🔒 MySQL connection closed
columns in get_raw_data_colums ['columns', 'row_hash']
Empty DataFrame
Columns: [city, price, prev_sold_date, bed, status, house_size, zip_code, brokered_by, street, state, acre_lot, bath]
Index: []


In [12]:
create_table("raw_data", df_new)
create_table("raw_dat_temporal", df_new)

port 8005
create_table ddl 
    CREATE TABLE IF NOT EXISTS `raw_data` (
      `id` BIGINT UNSIGNED NOT NULL AUTO_INCREMENT,
      `city` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `price` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `prev_sold_date` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `bed` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `status` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `house_size` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `zip_code` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `brokered_by` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `street` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `state` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `acre_lot` VARCHAR(255) NOT NULL DEFAULT '0' ,
  `bath` VARCHAR(255) NOT NULL DEFAULT '0' ,
      /* SHA2-256 -> 32 bytes binary; safer than MD5 for collision risk */
      `row_hash` BINARY(32)
        GENERATED ALWAYS AS (UNHEX(SHA2(CONCAT_WS('|',COALESCE(`city`,''), COALESCE(`price`,''), COALESCE(`prev_sold_date`,''), COALESCE(`bed`,''), COALESCE(`status`,''), COALESCE(`house_size`,

In [18]:
restart_response = requests.get(RESTART_URL)
print(f"All data fetched. Columns saved, restart response: {restart_response}")

All data fetched. Columns saved, restart response: <Response [200]>
